# [모듈 3.1] Code Pipeline 생성 하기
## 참고 자료
- https://docs.aws.amazon.com/codepipeline/latest/userguide/pipelines-create.html


# 1. 환경 설정 


In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [ ]:
%store -r code_pipeline_train_config_json_path
%store -r sm_pipeline_train_config_json_path

from common_utils import load_json

code_pipeline_train_dict = load_json(code_pipeline_train_config_json_path)
sm_pipeline_train_dict = load_json(sm_pipeline_train_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_train_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_train_dict, indent=2))



# 2. 파라미터 설정


In [ ]:
region = code_pipeline_train_dict["region"]
repository_name = code_pipeline_train_dict["train_code_repo_name"]
code_build_project_name = code_pipeline_train_dict["code_build_project_name"]
code_pipeline_name = code_pipeline_train_dict["code_pipeline_name"]
branch_name = code_pipeline_train_dict["branch_name"]
pipeline_role_name = code_pipeline_train_dict["code_pipeline_role_arn"]
bucket = code_pipeline_train_dict["bucket"]


# 3. 파이프라인 JSON 정의

In [ ]:
pipeline_dic = {
    "pipeline": {
        "name": code_pipeline_name,
        "roleArn": pipeline_role_name,
        "artifactStore": {
            "type": "S3",
            "location": bucket
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": branch_name,
                            "OutputArtifactFormat": "CODE_ZIP",
                            "PollForSourceChanges": "false",
                            "RepositoryName": repository_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "inputArtifacts": [],
                        "region": region,
                        "namespace": "SourceVariables"
                    }
                ]
            },
            {
                "name": "Build",
                "actions": [
                    {
                        "name": "BuildAndExecuteSageMakerPipeline",
                        "actionTypeId": {
                            "category": "Build",
                            "owner": "AWS",
                            "provider": "CodeBuild",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "ProjectName": code_build_project_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "BuildArtifact"
                            }
                        ],
                        "inputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "region": region,
                        "namespace": "BuildVariables"
                    }
                ]
            }
        ],
        "version": 1
    },
}

## 3.1. Dic 포맷을 JSON 으로 변경

In [ ]:
import json

pipeline_json = json.dumps(pipeline_dic)
json.loads(pipeline_json)

## 3.2. JSON 파일 저장

In [ ]:
json_file_path = 'src/hello_pipeline.json'
with open(json_file_path, "w") as outfile:
    outfile.write(pipeline_json)

In [ ]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

# 4. 파이프라인 실행

In [ ]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
echo $PWD
aws codepipeline create-pipeline --cli-input-json file://$json_file_path
# aws codepipeline create-pipeline --cli-input-json file://src/hello_pipeline.json

# 5. 코드 파이프라인 실행 상태 확인

In [ ]:
%%sh -s {code_pipeline_name}
code_pipeline_name=$1
echo $code_pipeline_name
aws codepipeline get-pipeline-state --name $code_pipeline_name

# 6. Code Pipeline 생성 확인

## 6.1. Code Pipeline 실행 확인

![ncf_code_pipeline.png](img/ncf_code_pipeline.png)

## 6.2. Code Pipeline Stage 확인

![ncf_code_pipeline.png](img/ncf_code_stages_pipeline.png)

## 6.3. Code Build 실행 로그 확인

![ncf_build_detail.png](img/ncf_build_detail.png)

## 6.4. SageMaker Pipleline 생성 확인

![ncf_train_sm_pipeline.png](img/ncf_train_sm_pipeline.png)

## 6.5. SageMaker NCF Pipeline Execution 확인

![ncf_train_sm_pipe_exec.png](img/ncf_train_sm_pipe_exec.png)

## 6.6. SageMaker Pipeline 실행 스텝 학인

![ncf_train_pipeline_steps.png](img/ncf_train_pipeline_steps.png)

## 6.7. Model Registry

![ncf_model_registry.png](img/ncf_model_registry.png)

## 6.8. Model Version in the Model Group

![ncf_model_version.png](img/ncf_model_version.png)

## 6.9. 등록된 추론 도커 컨테이너 및 모델 아티펙트

![model_version_setting_front.png](img/model_version_setting_front.png)
![model_version_container.png](img/model_version_container.png)